# 二次元移流方程式
$$\partial_t \phi + u\cdot \nabla \phi = 0$$
+ 矩形 の時間発展を追う
+ 時間差分はオイラー法
+ 空間差分は風上差分
+ $u_x, u_y > 0$ を仮定

## TODO
+ 境界を 0 と N+1 にしたいけどなんかないのかな
+ 時間差分の方法とか空間差分の方法とか検証したい
+ $u$ を非一様 (特に渦) にしたい

In [1]:
using Makie

In [2]:

Nx = 300; Ny = 150
ϕ= Array{Float64,2}(undef,Nx+2, Ny+2)
u = copy(ϕ)
v = copy(ϕ)
u .= 1
v .= 1
ϕ .= 0
ϕ[2:51,2:101] .= 1.

heatmap(ϕ[:,:])


In [3]:
function t1_x1_kazakami(ϕ,u,v;dx=1,dt=1e-1,T=500,step=10)
  Nx,Ny = size(ϕ)
  
  Bx = 2:Nx-1; By = 2:Ny-1
  dϕ = copy(ϕ)
  Lϕ = Array{Float64,3}(undef,Nx,Ny,div(length(0:dt:T),step)+1)
  
  
  for(i,t) in enumerate(0:dt:T)
    #Periodic Boundary Condition
    #boundary
    #edge
    for x in Bx
      ϕ[x,1] = ϕ[x,Ny-1]
      ϕ[x,Ny] = ϕ[x,2]
    end
    for y in By
      ϕ[1,y] = ϕ[Nx-1,y]
      ϕ[Nx,y] = ϕ[2,y]
    end
    
    #corner
    ϕ[1,1]=ϕ[Nx-1,Ny-1]
    ϕ[Nx,1]=ϕ[2,Ny-1]
    ϕ[1,Ny]=ϕ[Nx-1,2]
    ϕ[Nx,Ny]=ϕ[2,2]
    
    for y in By, x in Bx
      dϕ[x,y] = u[x,y] * 1/dx * (ϕ[x,y]-ϕ[x-1,y])+ v[x,y]  * 1/dx * (ϕ[x,y]-ϕ[x,y-1])      
    end
    
    for y in By, x in Bx
      ϕ[x,y] -= dt*dϕ[x,y]
    end
    
    if i%step ==1
      Lϕ[:,:,div(i,step)+1] .= ϕ
    end
    if i%(div(length(0:dt:T),10)) ==1
      println(i, "//",length(0:dt:T) )
    end
  end
  return Lϕ
end

t1_x1_kazakami (generic function with 1 method)

In [4]:
@time L = t1_x1_kazakami(ϕ,u,v);

1//5001
501//5001
1001//5001
1501//5001
2001//5001
2501//5001
3001//5001
3501//5001
4001//5001
4501//5001
5001//5001
  1.249782 seconds (850.46 k allocations: 217.688 MiB, 2.18% gc time)


In [5]:
"動画作成"
function movie(ϕ; fname = "test.mp4")
  scene = Scene(resolution = (1000,500))
  mystep = Node(1)
  
  scene = heatmap!(
    scene,
    lift(i -> ϕ[:,:,i],mystep),
    )
   
  
  record(scene, fname, 1:size(ϕ)[3]) do i
    mystep[] = i
  end
end

movie

In [6]:
@time movie(L;fname = "2D_1.mp4")

 44.413350 seconds (43.10 M allocations: 3.166 GiB, 1.65% gc time)


"2D_1.mp4"